In [2]:
from __future__ import print_function
import argparse
import torch
import copy
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from mftma.utils.make_manifold_data import make_manifold_data
from mftma.utils.activation_extractor import extractor
from neural_manifold_utils import train, test, save_dict, sub_data, create_manifold_data
from torch.utils.data.sampler import SubsetRandomSampler
import os, sys
import socket
from datetime import datetime
import getpass
import numpy as np 
print('__cuda available ',torch.cuda.is_available())
print('__Python VERSION:', sys.version)
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())

import mat73
#from mftma.manifold_analysis_correlation import manifold_analysis_corr
#from mftma.utils.make_manifold_data import make_manifold_data
from mftma.utils.activation_extractor import extractor
#from mftma.utils.analyze_pytorch import analyze


__cuda available  True
__Python VERSION: 3.6.10 (default, Apr 23 2020, 15:24:07) 
[GCC 8.3.0]
__CUDNN VERSION: 7603
__Number CUDA Devices: 4


In [4]:
user=getpass.getuser()
print(user)
if user=='eghbalhosseini':
    save_dir='/Users/eghbalhosseini/MyData/neural_manifolds/network_training_on_synthetic/'
    data_dir='/Users/eghbalhosseini/MyData/neural_manifolds/synthetic_datasets/'
elif user=='ehoseini':
    save_dir='/om/user/ehoseini/MyData/neural_manifolds/network_training_on_synthetic/'
    data_dir='/om/user/ehoseini/MyData/neural_manifolds/synthetic_datasets/'
datafile="synth_tree_nobj_50000_nclass_50_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat"

batch_size_train=64
batch_size_test=64
resize = True
epochs=2
exm_per_class=10
save_epochs=False
momentum=0.5
lr=0.01
gamma=0.7
log_interval=75
test_split = .2
shuffle_dataset = True
random_seed = 1



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = sub_data(data_path=os.path.join(data_dir, datafile))
if not os.path.exists(save_dir):
        os.makedirs(save_dir)
# Creating data indices for training and test splits:
random_seed=1
test_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
# Define transforms
transform_2D = transforms.Compose([
    transforms.ToTensor(),
])
# train_dataset = ApplyTransform(train_dataset, transform=train_transform)
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size_train, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size_test,sampler=test_sampler)

ehoseini
<HDF5 dataset "data": shape (100124,), type "<f8"> is not a matlab type
<HDF5 dataset "ir": shape (100124,), type "<u8"> is not a matlab type
<HDF5 dataset "jc": shape (50064,), type "<u8"> is not a matlab type
data type not supported: graph, uint32
data type not supported: graph, uint32


In [5]:
num_hierarchy=len(dataset.hierarchical_target)
hier_classes = [x.astype(int) for x in dataset.hierarchical_target]
hier_n_class = [int(max(x) + 1) for x in hier_classes]
hier_dataset=[]
for idx, x in enumerate(hier_classes):
    dat_mfmta = []
    dat_mtmfa = copy.deepcopy(dataset)
    dat_mtmfa.targets = hier_classes[idx]
    hier_dataset.append(copy.deepcopy(dat_mtmfa))

hier_sample_mtmfa= [create_manifold_data(x, hier_n_class[idx], exm_per_class, seed=0) for idx, x in enumerate(hier_dataset)]
hier_sample_mtmfa=[[d.to(device) for d in data] for data in hier_sample_mtmfa]


In [6]:
train_spec = {'train_batch_size': batch_size_train,
                  'test_batch_size': batch_size_test,
                  'num_epochs': epochs,
                  'structure': dataset.structure,
                  'n_class': dataset.n_class,
                  'exm_per_class': dataset.exm_per_class,
                  'beta': dataset.beta,
                  'sigma': dataset.sigma,
                  'norm': dataset.is_norm,
                  'log_interval':log_interval,
                  'is_cuda':torch.cuda.is_available()
                  }

    # build model

model = models.vgg16(num_classes=dataset.n_class)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    # create train log for tensorflow
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(save_dir, 'runs', current_time + '_' + socket.gethostname())
    #writer = SummaryWriter(log_dir=log_dir)
    #writer.add_hparams(hparam_dict=train_spec, metric_dict={})
    # do the training :
train_test_accu=[]


In [7]:
data_ = {'activations_cell': [],
             'train_spec': train_spec,
             'train_test_data': [],
             'train_success':False
             }
model_dir=save_dir+'train_VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
if not os.path.exists(model_dir):
        os.makedirs(model_dir)

In [8]:
len(train_loader.sampler)

40000

In [10]:
for epoch in range(1, epochs + 1):
        data_ = {'activations_cell': [],
                 'train_spec': train_spec,
                 'train_accuracy': [],
                 'test_accuracy':[],
                 'train_success': False,
                 'epoch':epoch}
        train_test_data = []
        epoch_dat = train(epoch, model, device, train_loader, test_loader, optimizer, train_spec)
        test_accuracy = test(model, device, test_loader, epoch)
        if test_accuracy > 70:
            train_success = True
        else:
            train_success = False
        # extract activation
        model = model.eval()
        activations_cell = [extractor(model, x, layer_types=['Conv2d', 'Linear']) for x in hier_sample_mtmfa]
        data_['train_accuracy']=epoch_dat['train_acc']
        data_['test_accuray']=test_accuracy
        data_['activations_cell'] = activations_cell
        data_['train_success'] = train_success
        epoch_save_path = os.path.join(model_dir, 'train_epoch_' + str(epoch))
        save_dict(data_, epoch_save_path)
        if train_success:
            print('successful training')
            break

Train Epoch: [4800/50000 (12%)]\Loss: 3.915713, Train Accuracy: (2%)
Train Epoch: [9600/50000 (24%)]\Loss: 3.911180, Train Accuracy: (3%)
Train Epoch: [14400/50000 (36%)]\Loss: 3.912959, Train Accuracy: (2%)
Train Epoch: [19200/50000 (48%)]\Loss: 3.910235, Train Accuracy: (2%)
Train Epoch: [24000/50000 (60%)]\Loss: 3.916163, Train Accuracy: (0%)
Train Epoch: [28800/50000 (72%)]\Loss: 3.909770, Train Accuracy: (8%)
Train Epoch: [33600/50000 (84%)]\Loss: 3.912319, Train Accuracy: (0%)
Train Epoch: [38400/50000 (96%)]\Loss: 3.915257, Train Accuracy: (3%)
couldnt reformat the data

Test set epoch 1: Average loss: 0.7825, Accuracy: 184/10000 (2%)

Train Epoch: [4800/50000 (12%)]\Loss: 3.910552, Train Accuracy: (2%)
Train Epoch: [9600/50000 (24%)]\Loss: 3.913121, Train Accuracy: (2%)
Train Epoch: [14400/50000 (36%)]\Loss: 3.906315, Train Accuracy: (2%)
Train Epoch: [19200/50000 (48%)]\Loss: 3.913738, Train Accuracy: (3%)
Train Epoch: [24000/50000 (60%)]\Loss: 3.909599, Train Accuracy: (2%)
T

In [ ]:
activations_cell=[extractor(model,x,layer_types=['Conv2d', 'Linear']) for x in hier_sample_mtmfa]

In [ ]:
m=(activations_cell[0]['layer_0_Input'][1])

In [ ]:
print("%d bytes" % (17*10*n.size * n.itemsize+17*50*m.size * m.itemsize))

In [ ]:
def train(epoch,model, device, train_loader,test_loader, optimizer,train_spec):
    model.train()
    test_accuracies = []
    train_accuracies = []
    fcs=[]
    targets=[]
    batchs=[]
    log_interval=train_spec['log_interval']
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = torch.nn.functional.log_softmax(output, dim=1)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx % log_interval == 0) & (batch_idx!=0):
            pred = output.argmax(dim=1, keepdim=True)
            correct = pred.eq(target.view_as(pred)).sum().item()
            accuracy_train = (100. * correct / len(target))
            train_accuracies.append(accuracy_train)
            print('Train Epoch: [{}/{} ({:.0f}%)]\Loss: {:.6f}, Train Accuracy: ({:.0f}%)'.format(
                 batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(),
                 100. * correct / len(target)))
    epoch_dat = {
        "fc": fcs,
        "target": target,
        "batch":  batchs,
        "epoch": epoch,
        "test_acc": test_accuracies,
        "train_acc": train_accuracies}

    # if is_cuda:/
    if train_spec['is_cuda']:
        try :
            epoch_dat['test_acc'] = np.concatenate(epoch_dat['test_acc'])
            epoch_dat['train_acc'] = np.concatenate(epoch_dat['train_acc'])
            epoch_dat['fc'] = np.concatenate(epoch_dat['fc'], axis=0)
            epoch_dat['target'] = np.concatenate(epoch_dat['target'])
            epoch_dat['batch'] = np.concatenate(epoch_dat['batch'])
        except: 
            print('couldnt reformat the data')
    return epoch_dat
